In [0]:
from google.colab import drive
import os

Mount a directory

Load and store pickle files

In [0]:
def mountDir(dir, subdir = '', path = 'drive/My Drive/'):
  
    drive.mount('/content/drive')
    print(os.getcwd())
    if subdir == '':
        os.chdir(path + dir)
    else:
        
        os.chdir(path + dir + '/' + subdir)
    print(os.getcwd())
    # !ls -la

def storeData(data, filename, ext = '.pkl'):

    pickle.dump(data, open(filename + ext, 'wb') )  
    print(filename + ext + ' saved!')

def loadData(filename, ext = '.pkl'):     

    filename += ext
    data = pickle.load(open(filename, 'rb'))  
    print(filename, ' loaded!')
    
    return data


In [0]:
mountDir('IRProject')

Fetch data directly from link

In [0]:
# !wget http://vision.cs.utexas.edu/projects/finegrained/utzap50k/ut-zap50k-images.zip
# !wget ftp://ftp.inrialpes.fr/pub/lear/douze/data/jpg1.tar.gz

In [0]:
# %cd shoe_dataset
# !unzip ut-zap50k-images.zip
# %cd ..
# %cd holiday_dataset
# !tar -xvf jpg1.tar.gz -C jpg
# %cd jpg
# !ls

In [0]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import cv2
from google.colab.patches import cv2_imshow # because cv2.imshow does'nt work in colab
from matplotlib import pyplot as plt

In [0]:
def build_paths(title):
    paths = []
    for (dirpath, dirnames, filenames) in tqdm(os.walk(str(os.getcwd())+'/'+title+'/')):
        for i in filenames:
            paths.append(str(dirpath)+str("/")+i)
    return paths

In [0]:
def getPaths(load = True, shoe = True, holiday = True):
    if load:
        if shoe:
            path_shoe = loadData('path_shoe', '.list')
        if holiday:
            path_holiday = loadData('path_holiday', '.list')
    else:
        if shoe:
            path_shoe = build_paths('shoe_dataset')
            storeData(path_shoe, 'path_shoe', '.list')
        if holiday:            
            path_holiday = build_paths('holiday_dataset')
            storeData(path_holiday, 'path_holiday', '.list')
    if shoe:
        if holiday:
            return path_shoe, path_holiday
        return path_shoe
    if holiday:
        return path_holiday
    return None

In [0]:
def getImages(load = True, shoe = True, holiday = True):
    s_color = list()
    s_grey = list()
    h_color = list()
    h_grey = list()
    if load:
        if shoe:
            s_color = loadData('s_color_raw', '.list')
            s_grey = loadData('s_grey_raw', '.list')
        if holiday:
            h_color = loadData('h_color_raw', '.list')
            h_grey = loadData('h_grey_raw', '.list')
    else:
        if shoe:
            for i in tqdm(range(len(path_shoe))):
                img = cv2.imread(path_shoe[i])
                s_color.append(img)
                s_grey.append(cv2.cvtColor(img,cv2.COLOR_BGR2GRAY))
            storeData(s_color, 's_color_raw', '.list')
            storeData(s_grey, 's_grey_raw', '.list')
        if holiday:            
            for i in tqdm(range(len(path_holiday))):
                img = cv2.imread(path_holiday[i])
                r = 240/min(img.shape[:2])
                img = cv2.resize(img, (0,0), fx=r, fy=r)
                # cv2_imshow(img)
                x = int((img.shape[0] - 240)/2)
                y = int((img.shape[1] - 240)/2)
                img = img[x:x+240,y:y+240]
                filename = 'holiday_dataset2/' + path_holiday[i].split('/')[-1]
                cv2.imwrite(filename, img)
                h_color.append(img)
                h_grey.append(cv2.cvtColor(img,cv2.COLOR_BGR2GRAY))
            storeData(h_color, 'h_color_raw', '.list')
            storeData(h_grey, 'h_grey_raw', '.list')
    if shoe:
        if holiday:
            return s_color, s_grey, h_color, h_grey
        return s_color, s_grey
    if holiday:
        return h_color, h_grey
    return None

In [0]:
# path_shoe, path_holiday = getPaths()
s_color, s_grey, h_color, h_grey = getImages()

s_color_raw.list  loaded!
s_grey_raw.list  loaded!
h_color_raw.list  loaded!
h_grey_raw.list  loaded!


In [0]:
# df = pd.DataFrame({'path': path_shoe, 'color': s_color, 'grey': s_grey})
# df.head()
hdf = pd.DataFrame({'path': path_holiday, 'color': h_color, 'grey': h_grey})
hdf.head()

Generate Feature Sets

In [0]:
def histogram(image, bins = 10):
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

In [0]:
def hu_moments(image):
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

In [0]:
!pip install mahotas
import mahotas
# feature-descriptor-2: Haralick Texture
def haralick(image):
    # convert the image to grayscale
    # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(image).mean(axis=0)
    # return the result
    return haralick

In [0]:
from skimage.feature import hog

def hog_f(image):
    fd, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True, multichannel=True)
    return fd

In [0]:
def preprocess_image(img):
    # img = cv2.imread(path)                 #original image
    img2 = np.invert(img)                  #invert image
    gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    img2 = cv2.Canny(gray, threshold1=30, threshold2=100)    #edge detection
    img2 = cv2.GaussianBlur(img2,(5,5),cv2.BORDER_DEFAULT) #gaussian blur 
    ret, img2 = cv2.threshold(img2, 1, 255, cv2.THRESH_BINARY)     #thresholding 
    img2 = cv2.adaptiveThreshold(img2 ,255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 255, 1)
    img2 = np.invert(img2)
    temp = tuple(img2 for i in range(3))
    temp = np.dstack(temp)
    image = cv2.subtract(img, temp)
    return image

In [0]:
def getFeatures(images, bins = 10):
    hist = list()
    hu_moment = list()
    haral = list()
    hogf = list()
    orbf = list()
    for i in tqdm(range(len(images))):
        image = images[i]
        grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        grey = cv2.GaussianBlur(grey,(5,5),cv2.BORDER_DEFAULT) #gaussian blur 

        hu_moment.append(hu_moments(grey))
        haral.append(haralick(grey))

        image = preprocess_image(images[i])

        hist.append(histogram(image, bins))
        hogf.append(hog_f(image))
        orb = cv2.ORB_create()

        # find the keypoints with ORB
        kp = orb.detect(image,None)
        # compute the descriptors with ORB
        kp, des = orb.compute(image, kp)
        if des is None:
            orbf.append(np.array([[0]*32],dtype = 'uint8').flatten())
        else:
            orbf.append(des.flatten())       
    storeData(hist, 'h_hist', '.list')
    storeData(hogf, 'h_hog', '.list')
    storeData(haral, 'h_haral', '.list')
    storeData(hu_moment, 'h_hu', '.list')
    storeData(orbf, 'h_orb', '.list')
    return True

In [0]:
fl = getFeatures(hdf.color.values)

100%|██████████| 812/812 [03:26<00:00,  3.94it/s]


h_hist.list saved!
h_hog.list saved!
h_haral.list saved!
h_hu.list saved!
h_orb.list saved!


In [0]:
hist = loadData('s_hist', '.list')
hogf = loadData('s_hog', '.list')
haral = loadData('s_haral', '.list')
hu_moment = loadData('s_hu', '.list')
orbf = loadData('s_orb', '.list')

s_hist.list  loaded!
s_hog.list  loaded!
s_haral.list  loaded!
s_hu.list  loaded!
s_orb.list  loaded!


In [0]:
type(hist), type(hogf), type(haral), type(hu_moment), type(orbf)

(list, list, list, list, list)

get histogram after segmentation

In [0]:
def Histogram(image, mask):
    hist = cv2.calcHist([image], [0, 1, 2], mask, (8, 12, 3), [0, 180, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

def get_feature(images):
    featlist = list()
    for i in tqdm(range(len(images))):
        image = images[i]
        # image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        feature = []
        (h, w) = image.shape[:2]
        (cX, cY) = (int(w * 0.5), int(h * 0.5))
        segments = [(0, cX, 0, cY), (cX, w, 0, cY), (cX, w, cY, h),(0, cX, cY, h)]
        (axesX, axesY) = (int(w * 0.75) // 2, int(h * 0.75) // 2)
            
        ellipMask = np.zeros(image.shape[:2], dtype = "uint8")
        cornerMask = np.zeros(image.shape[:2], dtype = "uint8")
        cv2.ellipse(ellipMask, (cX, cY), (axesX, axesY), 0, 0, 360, 255, -1)
            
        for (startX, endX, startY, endY) in segments:
            cv2.rectangle(cornerMask, (startX, startY), (endX, endY), 255, -1)
            cornerMask = cv2.subtract(cornerMask, ellipMask)
            hist = Histogram(image, cornerMask)
            feature.extend(hist)
        hist = Histogram(image, ellipMask)
        feature.extend(hist)
        featlist.append(feature)
    storeData(featlist, 'h_seghist', '.list')
    return featlist

In [0]:
seghist = get_feature(hdf.color.values)

100%|██████████| 812/812 [00:00<00:00, 1197.74it/s]


h_seghist.list saved!


In [0]:
images = df.color.values

In [0]:
# !pip install PyWavelets
import pywt
# coeffs2 = pywt.dwt2(df.color.values[0], 'bior1.3')
# LL, (LH, HL, HH) = coeffs2
# images[0].shape
# sta = np.hstack([LH, HL, HH])
# # cv2_imshow(cv2.subtract(images[0], ))
# images[0].shape, sta.shape, LH.shape

Holiday Dataset

In [0]:
# import numpy as np
# from numpy import linalg as LA
# # from extract_cnn_vgg16_keras import VGGNet
# from keras.applications.vgg16 import VGG16
# from keras.preprocessing import image
# from keras.applications.vgg16 import preprocess_input

# class VGGNet:
#     def __init__(self):
#         # weights: 'imagenet'
#         # pooling: 'max' or 'avg'
#         # input_shape: (width, height, 3), width and height should >= 48
#         self.input_shape = (102, 136, 3)
#         self.weight = 'imagenet'
#         self.pooling = 'max'
#         self.model = VGG16(weights = self.weight, input_shape = (self.input_shape[0], self.input_shape[1], self.input_shape[2]), pooling = self.pooling, include_top = False)
#         self.model.predict(np.zeros((1, 102, 136 , 3)))

#     '''
#     Use vgg16 model to extract features
#     Output normalized feature vector
#     '''
#     def extract_feat(self, img_path):
#         # img=cv2.imread(img_path)
#         img = image.load_img(img_path, target_size=(self.input_shape[0], self.input_shape[1]))
#         img = image.img_to_array(img)
#         # img=img.resize(224,224)
#         img = np.expand_dims(img, axis=0)
#         img = preprocess_input(img)
#         feat = self.model.predict(img)
#         norm_feat = feat[0]/LA.norm(feat[0])
#         return norm_feat

In [0]:
# model=VGGNet()

In [0]:
# print("####features extraction process begins ####")
# features=[]
# for i in tqdm(range(len(path))):
  
#   queryFeat = model.extract_feat(path[i])
#   features.append(queryFeat)

# print("####features created ####")